In [1]:
import os, pandas as pd, numpy as np

In [2]:
import matplotlib.pyplot as plt, seaborn as sns
sns.set_style('darkgrid')
%matplotlib notebook

In [4]:
os.listdir('../../data/codedtruth')

['.DS_Store',
 'paper_metadata.csv',
 'papertags_how_withwhom_final.csv',
 'papertags_what_final.csv',
 'truth_metadata.csv',
 'truth_overflow.csv',
 'truth_ratings.csv',
 'v_11_coded_final.csv',
 'v_16_coded_final.csv',
 'v_19_coded_final.csv',
 'v_6_coded_final.csv',
 'v_8345etseq_final.csv',
 'v_8780etseq_final.csv']

In [22]:
df = pd.read_csv('../../data/codedtruth/paper_metadata.csv', encoding='latin1')

In [11]:
df.head(1)

,PaperID,Title,Authors,Venue,Year,NumberOfPages,AcadVsInd,IndTrack,Summary
0,1,Software traceability with topic modeling,"Hazeline U. Asuncion, Arthur U. Asuncion, Rich...",ICSE,2010.0,10,Academic,No,A method for automatically recovering software...


In [25]:
df.tail(1)

,PaperID,Title,Authors,Venue,Year,NumberOfPages,AcadVsInd,IndTrack,Summary
434,435,DIGS: A Framework for Discovering Goals for Se...,"Maria Riaz, Jonathan Stallings, Munindar P. Si...",ESEM,2016,10,Academic,No,A framework for supporting a systematic and co...


### Chapter 2, Section 1

In [29]:
df.groupby(['Year','Venue']).count()[['PaperID']].unstack()
# note that the values are converted to floats due to the NaN

PaperID                            
Venue ESEC/FSE ESEM  FSE  ICSE    RE REFSQ
Year                                      
2010       NaN  3.0  NaN   5.0  38.0  15.0
2011       3.0  1.0  NaN   4.0  33.0  10.0
2012       NaN  4.0  1.0   3.0  35.0  14.0
2013       4.0  2.0  NaN   9.0  34.0  23.0
2014       NaN  7.0  NaN  10.0  44.0  18.0
2015       1.0  1.0  NaN   1.0  28.0  18.0
2016       NaN  4.0  7.0   7.0  33.0  15.0

In [31]:
df.groupby(['Year','Venue']).count(
)[['PaperID']].unstack().sum(axis=0).astype(int)

         Venue   
PaperID  ESEC/FSE      8
         ESEM         22
         FSE           8
         ICSE         39
         RE          245
         REFSQ       113
dtype: int64

In [33]:
df.groupby(['Year','Venue']).count(
)[['PaperID']].unstack().sum(axis=1).astype(int)

Year
2010    61
2011    51
2012    57
2013    72
2014    79
2015    49
2016    66
dtype: int64

In [34]:
df.groupby(['AcadVsInd','IndTrack']).count()[['PaperID']].unstack()

PaperID    
IndTrack       No Yes
AcadVsInd            
Academic      296  20
Industry        7  28
Mixed          43  41

In [36]:
df.groupby(['AcadVsInd','IndTrack']).count(
)[['PaperID']].unstack().sum(axis=0)

         IndTrack
PaperID  No          346
         Yes          89
dtype: int64

In [37]:
df.groupby(['AcadVsInd','IndTrack']).count(
)[['PaperID']].unstack().sum(axis=1)

AcadVsInd
Academic    316
Industry     35
Mixed        84
dtype: int64

In [91]:
df.Summary.values

array(['A method for automatically recovering software traceability links between various software artifacts based on topic modelling (requirements, design, code, bug reports, test cases)',
       'A set of two techniques for improving the quality of traces generated between regulatory standards and product level requirements',
       'A case study on evaluating a given technique for identifying and prioritising stakeholders in order to involve the right stakeholders in system analysis',
       'An experience report on the development of a methodology and tools for the formalization and subsequent validation of specifications in a project for the public administration',
       'A document-driven study on the relevancy of clones in industrial requirements specifications and of clone detection as a requirements quality assurance technique',
       'A case study on the effects of a highly ambiguous requirements document on project success',
       'A data-driven study on the importance of

In [39]:
len(set([tuple(x.split(' ')[:3]) for x in df.Summary.values]))

50

### Chapter 2, Section 3

In [70]:
def add_split_levels(df, level_4=False):
    df['level_1'], df['level_2'], df['level_3'] = list(zip(*[(x[0], x[1], x[2]) 
                                                            for x in df.Tag.str.split(':')]))
    if level_4:
        df['level_3'], df['level_4'] = list(zip(*[(x[0], x[1] if len(x) > 1 else '')
                                             for x in df['level_3'].str.split('_')]))
    return df

In [72]:
how = pd.read_csv('../../data/codedtruth/papertags_how_withwhom_final.csv')
how = how[~how.Tag.str.startswith('withwhom')]
how = add_split_levels(how)

In [73]:
withwhom = pd.read_csv('../../data/codedtruth/papertags_how_withwhom_final.csv')
withwhom = withwhom[withwhom.Tag.str.startswith('withwhom')]
withwhom = add_split_levels(withwhom)

In [75]:
what = pd.read_csv('../data/codedtruth/papertags_what_final.csv')
what = add_split_levels(what, level_4=True)

In [77]:
levels = [f'level_{x}' for x in range(1,5)]

In [91]:
how.groupby(levels[:-1]).count()[['Tag']].sort_index(level=1)

Tag
level_1 level_2     level_3           
how     engineering methodology    177
                    reference        5
                    technology      33
        perspective experience      38
                    opinion         11
                    philosophy       1
                    review          14
        science     interrogation   45
                    intervention    39
                    observation     77

In [84]:
print(how.groupby(levels[:-1]).count()[['Tag']].to_latex())

\begin{tabular}{lllr}
\toprule
    &             &            &  Tag \\
level\_1 & level\_2 & level\_3 &      \\
\midrule
how & engineering & methodology &  177 \\
    &             & reference &    5 \\
    &             & technology &   33 \\
    & perspective & experience &   38 \\
    &             & opinion &   11 \\
    &             & philosophy &    1 \\
    &             & review &   14 \\
    & science & interrogation &   45 \\
    &             & intervention &   39 \\
    &             & observation &   77 \\
\bottomrule
\end{tabular}



In [98]:
print(withwhom.groupby(levels[:-1]).count()[[]].to_latex())

\begin{tabular}{lll}
\toprule
         &           &           \\
level\_1 & level\_2 & level\_3 \\
\midrule
withwhom & laypeople & others \\
         &           & students \\
         & professionals & academics \\
         &           & practitioners \\
\bottomrule
\end{tabular}



In [85]:
print(withwhom.groupby(levels[:-1]).count()[['Tag']].to_latex())

\begin{tabular}{lllr}
\toprule
         &           &           &  Tag \\
level\_1 & level\_2 & level\_3 &      \\
\midrule
withwhom & laypeople & others &    1 \\
         &           & students &   28 \\
         & professionals & academics &    2 \\
         &           & practitioners &   30 \\
\bottomrule
\end{tabular}



In [96]:
print(what.groupby(levels).count()[[]].to_latex())

\begin{tabular}{llll}
\toprule
     &           &          & {} \\
level\_1 & level\_2 & level\_3 & level\_4 \\
\midrule
what & challenge & content & all \\
     &           &          & completeness \\
     &           &          & consistency \\
     &           &          & feasibility \\
     &           &          & traceability \\
     &           &          & unambiguousness \\
     &           &          & understandability \\
     &           & context & regulation \\
     &           &          & uncertainty \\
     &           & failure & {} \\
     &           & people & collaboration \\
     &           &          & communication \\
     &           &          & skills \\
     &           &          & subjectivity \\
     &           & problem & {} \\
     &           & process & automation \\
     &           &          & deciding \\
     &           &          & formalization \\
     &           &          & improving \\
     &           &          & prioritization \\
  